In [34]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup


In [5]:
category2={
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [96]:
categories = ["society", "politics", "economic", "foreign", "culture", "entertain", "sports", "digital"]

In [41]:
category, date, page = "digital", 20201108, 1

url = "https://news.daum.net/breakingnews/{}?page={}&regDate={}".format(category, page, date)

In [42]:
driver = webdriver.Chrome()
driver.get(url)

In [52]:
articles = driver.find_elements_by_css_selector(
    "div.box_etc > ul > li > div > strong > a")
len(articles)
#mArticle > div.box_etc > ul > li:nth-child(1) > div > strong > a

15

In [54]:
articles[1].text

'펀진, 인식률 99.97% 얼굴인식 알고리즘 KISA 인증 획득'

In [55]:
title = articles[0].text
link = articles[0].get_attribute("href")

title, link

('DIP-경북대, 빅데이터 생태계 활성화 협약', 'https://v.daum.net/v/20201108220118834')

In [66]:
ls = []
for article in articles:
    ls.append({
        "title": article.text,
        "link": article.get_attribute("href"),
    })
    
main_df = pd.DataFrame(ls)

In [74]:
main_df["link"][3]

'https://v.daum.net/v/20201108214201628'

In [85]:
response = requests.get(main_df["link"][5])
response

<Response [200]>

In [86]:
dom = BeautifulSoup(response.content, "html.parser")
content = dom.select_one("#harmonyContainer > section").text.replace('\n', '')
#harmonyContainer > section > p:nth-child(2)
content

'[경향신문] 조 바이든 미국 민주당 대선후보가 2020년 미 대선에서 승리하며 미국 ‘빅테크 업계’가 긴장하고 있다. 바이든 당선자가 이끌 미국의 새 행정부가 정보기술(IT) 대기업들에 대한 반독점 정책을 강화해 규제가 강화될 것이라는 우려가 큰 상황에서 민주당이 실리콘밸리와 돈독한 관계를 유지해온 만큼 최악의 상황까지는 가지 않을 것이라는 일말의 기대도 제기된다. 바이든 당선자가 주요 공약으로 내세운 법인세 강화도 거대 ‘IT 공룡’들엔 악재가 될 것으로 전망된다.8일 제46대 미국 대통령에 바이든 후보가 당선되면서 전통적으로 민주당을 지지해온 미국 IT업계가 들썩이고 있다. 가장 관심이 모아지는 이슈는 구글·페이스북·아마존·애플 등 IT ‘빅4’ 기업들에 대한 반독점 규제다.바이든 당선자가 이끄는 민주당은 거대 IT 기업들의 반독점 이슈를 눈여겨보고 있다. 지난달 미국 하원 법사위원회 소속 반독점위원회는 구글·애플·페이스북·아마존의 영향력 남용과 거대 플랫폼의 구조적 분리, 반독점법 강화 등을 담은 보고서를 내놨다. 바이든 당선자 역시 과거 언론 인터뷰에서 페이스북 등 실리콘밸리의 거대 IT 기업 해체 논의에 대해 면밀히 검토해봐야 할 문제라고 언급하는 등 IT 공룡들에 대한 반독점 규제를 지지해왔다. 여기에 코로나19로 빈익빈 부익부가 심화되는 상황에서 빅테크 기업들이 부를 독점하는 대상으로 인식되면서 새로 출범할 바이든 행정부가 이들 기업에 대한 규제 강도를 높일 것이라는 관측이 나온다.인터넷 기업들의 책임도 강화될 것으로 전망된다. 바이든 당선자는 올 1월 뉴욕타임스 인터뷰에서 온라인 플랫폼 기업이 이용자가 올린 게시물에 대한 법적 책임을 면제받도록 하는 ‘통신품위법(CDA) 230조’를 폐지해야 한다고 주장했다. 기존에는 구글과 페이스북 등 소셜미디어를 운영하는 기업들이 자사 플랫폼에 게재되는 허위 또는 명예훼손 콘텐츠에 대한 책임을 지지 않았지만 향후 면책 범위가 축소되면 가짜뉴스와 명예훼손성 게시글에 대한 소셜 기업들의 책임 부담이 발생할 수 

In [98]:
def get_articles(category, date, page):
    url = "https://news.daum.net/breakingnews/{}?page={}&regDate={}".format(category, page, date)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector("div.box_etc > ul > li > div > strong > a")
    
    ls = []
    for article in articles:
        link = article.get_attribute("href")
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#harmonyContainer > section").text.replace('\n', '').replace("\'", '')
        ls.append({
            "title": article.text,
            "link": link,
            "content": content,
            "category": category,
        })
    driver.quit()
    return pd.DataFrame(ls)

In [99]:
dfs = []
date = 20201108
for category in categories:
    print(category, end=' ')
    for page in range(1, 6):
        print(page, end=' ')
        df = get_articles(category, date, page)
        dfs.append(df)
    print()
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

society 1 2 3 4 5 
politics 1 2 3 4 5 
economic 1 2 3 4 5 
foreign 1 2 3 4 5 
culture 1 2 3 4 5 
entertain 1 2 3 4 5 
sports 1 2 3 4 5 
digital 1 2 3 4 5 


,title,link,content,category
595,"셀트리온 '휴미라' 바이오시밀러, 임상시험서 효과·안전성 입증",https://v.daum.net/v/20201108170902920,셀트리온이 미국 류머티즘학회(ACR)에서 자가면역질환 치료제 휴미라의 바이오시밀러(...,digital
596,[신SW상품대상추천작]갈래요모빌리티 '갈래요',https://v.daum.net/v/20201108170115737,갈래요모빌리티(대표 이계혁)는 택시를 이용하는 대리기사 간 온·오프라인연계(O2O)...,digital
597,[신SW상품대상추천작]이파피루스 '모터센스',https://v.daum.net/v/20201108170113735,이파피루스(대표 김정희)는 스마트 공장 구축 일환으로 모터 고장을 미리 예측하고 대...,digital
598,[신SW상품대상추천작]윅스콘 'WG-210',https://v.daum.net/v/20201108170109731,윅스콘(대표 박태준)은 통신망 장애와 품질 저하 시 자동으로 유·무선 통신망을 전환...,digital
599,'구글 인앱결제 방지법'에 추가규제 수두룩,https://v.daum.net/v/20201108165905643,이른바 ‘구글 인앱결제 방지법’ 마련을 위한 국회 공청회가 9일 열린다. 여당은 연...,digital


'            김종인 국민의힘 비대위원장이 지난 6일 오후 서울 마포구 제일라아트홀에서 열린 국민의힘 \'서울 시민후보 찾기 공청회\'에서 인사말을 하고 있다.(자료사진) ⓒ데일리안 홍금표 기자           김종인 국민의힘 비상대책위원장이 8일 안철수 국민의당 대표의 신당 창당 제안을 일축했다.김 위원장은 이날 서울 여의도 한 식당에서 중진의원들과의 만찬이 끝난 뒤 기자들과 만나 "혼자 하면 하는 것이지 어떻게 막겠나. 자기가 (창당) 한다는데"라고 말했다.앞서 안 대표는 지난 6일국민의힘과 국민의당 의원들이 함께 참여하는 국회의원 연구모임 \'국민미래포럼\' 비공개 간담회에서 "지지 기반을 넓히고 (야권을 향한) 비호감을 줄일 방법의 하나가 새로운 플랫폼, 사실 새로운 정당"이라고 말한 것으로 전해졌다.김 위원은 내년 4월 보궐선거와 관련해 중진 의원들과 어떤 이야기를 나눴는지를 묻는 질문에는 "우리가 내년에 보궐선거를 어떻게 대처해야 하는지 의견을 나눴다"며 "특별히 당부하거나 한 얘기는 없었다. 총력을 다해서 보궐선거를 맞이해야 한다"고 답했다.데일리안 이슬기 기자 (seulkee@dailian.co.kr)'